# Word2Vec Skip-gram# 

In [1]:
import numpy as np 
import string 
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:

def softmax(x): 
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x)) 
    return e_x / e_x.sum()

In [3]:
class word2vec(object): 
	def __init__(self): 
		self.N = 10
		self.X_train = [] 
		self.y_train = [] 
		self.window_size = 2
		self.alpha = 0.001
		self.words = [] 
		self.word_index = {} 

	def initialize(self,V,data): 
		self.V = V 
		self.W = np.random.uniform(-0.8, 0.8, (self.V, self.N)) 
		self.W1 = np.random.uniform(-0.8, 0.8, (self.N, self.V)) 
		
		self.words = data 
		for i in range(len(data)): 
			self.word_index[data[i]] = i 

	
	def feed_forward(self,X): 
		self.h = np.dot(self.W.T,X).reshape(self.N,1) 
		self.u = np.dot(self.W1.T,self.h) 
		#print(self.u) 
		self.y = softmax(self.u) 
		return self.y 
		
	def backpropagate(self,x,t): 
		e = self.y - np.asarray(t).reshape(self.V,1) 
		# e.shape is V x 1 
		dLdW1 = np.dot(self.h,e.T) 
		X = np.array(x).reshape(self.V,1) 
		dLdW = np.dot(X, np.dot(self.W1,e).T) 
		self.W1 -=  self.alpha*dLdW1 
		self.W -= self.alpha*dLdW 
		
	def train(self,epochs): 
		for x in range(1,epochs+1):		 
			self.loss = 0
			for j in range(len(self.X_train)): 
				self.feed_forward(self.X_train[j]) 
				self.backpropagate(self.X_train[j],self.y_train[j]) 
				C = 0
				for m in range(self.V): 
					if(self.y_train[j][m]): 
						self.loss += -1*self.u[m][0] 
						C += 1
				self.loss += C*np.log(np.sum(np.exp(self.u))) 
			print("epoch ",x, " loss = ",self.loss) 
			self.alpha *= 1/( (1+self.alpha*x) ) 
			
	def predict(self,word,number_of_predictions): 
		if word in self.words: 
			index = self.word_index[word] 
			X = [0 for i in range(self.V)] 
			X[index] = 1
			prediction = self.feed_forward(X) 
			output = {} 
			for i in range(self.V): 
				output[prediction[i][0]] = i 
			
			top_context_words = [] 
			for k in sorted(output,reverse=True): 
				top_context_words.append(self.words[output[k]]) 
				if(len(top_context_words)>=number_of_predictions): 
					break
	
			return top_context_words 
		else: 
			print("Word not found in dicitonary")

In [4]:
def preprocessing(corpus): 
	stop_words = set(stopwords.words('english'))	 
	training_data = [] 
	sentences = corpus.split(".") 
	for i in range(len(sentences)): 
		sentences[i] = sentences[i].strip() 
		sentence = sentences[i].split() 
		x = [word.strip(string.punctuation) for word in sentence if word not in stop_words] 
		x = [word.lower() for word in x] 
		training_data.append(x) 
	return training_data

In [5]:
def prepare_data_for_training(sentences,w2v): 
	data = {} 
	for sentence in sentences: 
		for word in sentence: 
			if word not in data: 
				data[word] = 1
			else: 
				data[word] += 1
	V = len(data) 
	data = sorted(list(data.keys())) 
	vocab = {} 
	for i in range(len(data)): 
		vocab[data[i]] = i 
	
	for sentence in sentences: 
		for i in range(len(sentence)): 
			center_word = [0 for x in range(V)] 
			center_word[vocab[sentence[i]]] = 1
			context = [0 for x in range(V)] 
			
			for j in range(i-w2v.window_size,i+w2v.window_size): 
				if i!=j and j>=0 and j<len(sentence): 
					context[vocab[sentence[j]]] += 1
			w2v.X_train.append(center_word) 
			w2v.y_train.append(context) 
	w2v.initialize(V,data) 

	return w2v.X_train,w2v.y_train,vocab

In [6]:

corpus = "In addition to these carefully designed methods,=. a word embedding can be learned as part of a deep learning model. This can be a slower approach, but tailors the model to a specific training dataset"
epochs = 50000

In [7]:
training_data = preprocessing(corpus)

In [8]:
training_data

[['in', 'addition', 'carefully', 'designed', 'methods'],
 ['word', 'embedding', 'learned', 'part', 'deep', 'learning', 'model'],
 ['this',
  'slower',
  'approach',
  'tailors',
  'model',
  'specific',
  'training',
  'dataset']]

In [9]:
w2v = word2vec()

In [10]:
prepare_data_for_training(training_data,w2v)

([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 

In [ ]:
w2v.train(epochs)

epoch  1  loss =  150.5959411968654
epoch  2  loss =  150.3964676037207
epoch  3  loss =  150.1977736083503
epoch  4  loss =  150.00004857570792
epoch  5  loss =  149.80347738234735
epoch  6  loss =  149.6082393909196
epoch  7  loss =  149.41450749607048
epoch  8  loss =  149.2224472539392
epoch  9  loss =  149.0322161054124
epoch  10  loss =  148.84396270104156
epoch  11  loss =  148.65782633317227
epoch  12  loss =  148.47393647845675
epoch  13  loss =  148.29241245159258
epoch  14  loss =  148.11336316893815
epoch  15  loss =  147.93688701865062
epoch  16  loss =  147.76307183222846
epoch  17  loss =  147.59199495085429
epoch  18  loss =  147.42372337873599
epoch  19  loss =  147.2583140147567
epoch  20  loss =  147.0958139531433
epoch  21  loss =  146.93626084354943
epoch  22  loss =  146.77968330088567
epoch  23  loss =  146.6261013553928
epoch  24  loss =  146.47552693381127
epoch  25  loss =  146.32796436300436
epoch  26  loss =  146.18341088802669
epoch  27  loss =  146.0418571

epoch  229  loss =  139.21999573551278
epoch  230  loss =  139.2136454466825
epoch  231  loss =  139.2073488156784
epoch  232  loss =  139.2011051735132
epoch  233  loss =  139.19491386213733
epoch  234  loss =  139.18877423421878
epoch  235  loss =  139.1826856529288
epoch  236  loss =  139.1766474917322
epoch  237  loss =  139.17065913418324
epoch  238  loss =  139.1647199737249
epoch  239  loss =  139.1588294134951
epoch  240  loss =  139.15298686613514
epoch  241  loss =  139.1471917536039
epoch  242  loss =  139.141443506996
epoch  243  loss =  139.1357415663645
epoch  244  loss =  139.13008538054638
epoch  245  loss =  139.1244744069943
epoch  246  loss =  139.11890811160944
epoch  247  loss =  139.1133859685808
epoch  248  loss =  139.10790746022593
epoch  249  loss =  139.1024720768369
epoch  250  loss =  139.0970793165286
epoch  251  loss =  139.0917286850912
epoch  252  loss =  139.08641969584525
epoch  253  loss =  139.08115186950067
epoch  254  loss =  139.07592473401843
ep

epoch  458  loss =  138.4830094967316
epoch  459  loss =  138.48139387924326
epoch  460  loss =  138.4797852644553
epoch  461  loss =  138.47818360707822
epoch  462  loss =  138.47658886221097
epoch  463  loss =  138.47500098533678
epoch  464  loss =  138.47341993231922
epoch  465  loss =  138.47184565939818
epoch  466  loss =  138.47027812318564
epoch  467  loss =  138.46871728066188
epoch  468  loss =  138.46716308917163
epoch  469  loss =  138.4656155064202
epoch  470  loss =  138.4640744904697
epoch  471  loss =  138.46253999973524
epoch  472  loss =  138.46101199298127
epoch  473  loss =  138.45949042931795
epoch  474  loss =  138.4579752681975
epoch  475  loss =  138.45646646941069
epoch  476  loss =  138.45496399308325
epoch  477  loss =  138.45346779967267
epoch  478  loss =  138.4519778499642
epoch  479  loss =  138.4504941050681
epoch  480  loss =  138.44901652641582
epoch  481  loss =  138.4475450757571
epoch  482  loss =  138.44607971515623
epoch  483  loss =  138.444620406

epoch  683  loss =  138.23854805463435
epoch  684  loss =  138.23782029141563
epoch  685  loss =  138.23709465301442
epoch  686  loss =  138.23637113015158
epoch  687  loss =  138.2356497136021
epoch  688  loss =  138.23493039419415
epoch  689  loss =  138.2342131628092
epoch  690  loss =  138.23349801038145
epoch  691  loss =  138.2327849278973
epoch  692  loss =  138.2320739063952
epoch  693  loss =  138.2313649369652
epoch  694  loss =  138.2306580107485
epoch  695  loss =  138.22995311893715
epoch  696  loss =  138.22925025277368
epoch  697  loss =  138.22854940355091
epoch  698  loss =  138.22785056261128
epoch  699  loss =  138.22715372134655
epoch  700  loss =  138.2264588711977
epoch  701  loss =  138.22576600365437
epoch  702  loss =  138.22507511025455
epoch  703  loss =  138.22438618258417
epoch  704  loss =  138.2236992122771
epoch  705  loss =  138.22301419101436
epoch  706  loss =  138.22233111052404
epoch  707  loss =  138.22164996258078
epoch  708  loss =  138.220970739

epoch  934  loss =  138.10478641211142
epoch  935  loss =  138.10439720495282
epoch  936  loss =  138.10400883011206
epoch  937  loss =  138.10362128492397
epoch  938  loss =  138.103234566735
epoch  939  loss =  138.10284867290272
epoch  940  loss =  138.102463600796
epoch  941  loss =  138.102079347795
epoch  942  loss =  138.10169591129076
epoch  943  loss =  138.10131328868565
epoch  944  loss =  138.1009314773929
epoch  945  loss =  138.1005504748366
epoch  946  loss =  138.10017027845183
epoch  947  loss =  138.0997908856845
epoch  948  loss =  138.09941229399126
epoch  949  loss =  138.09903450083948
epoch  950  loss =  138.0986575037072
epoch  951  loss =  138.09828130008307
epoch  952  loss =  138.09790588746623
epoch  953  loss =  138.09753126336628
epoch  954  loss =  138.09715742530344
epoch  955  loss =  138.09678437080814
epoch  956  loss =  138.09641209742125
epoch  957  loss =  138.09604060269376
epoch  958  loss =  138.09566988418698
epoch  959  loss =  138.09529993947

...

epoch  32329  loss =  137.75192215325373
epoch  32330  loss =  137.75192182926892
epoch  32331  loss =  137.75192150530395
epoch  32332  loss =  137.75192118135914
epoch  32333  loss =  137.7519208574343
epoch  32334  loss =  137.75192053352947
epoch  32335  loss =  137.75192020964474
epoch  32336  loss =  137.75191988578
epoch  32337  loss =  137.7519195619353
epoch  32338  loss =  137.75191923811067
epoch  32339  loss =  137.75191891430615
epoch  32340  loss =  137.75191859052146
epoch  32341  loss =  137.7519182667569
epoch  32342  loss =  137.7519179430124
epoch  32343  loss =  137.7519176192878
epoch  32344  loss =  137.75191729558327
epoch  32345  loss =  137.75191697189877
epoch  32346  loss =  137.75191664823436
epoch  32347  loss =  137.75191632458987
epoch  32348  loss =  137.75191600096548
epoch  32349  loss =  137.751915677361
epoch  32350  loss =  137.75191535377658
epoch  32351  loss =  137.75191503021216
epoch  32352  loss =  137.7519147066677
epoch  32353  loss =  137.7

epoch  32547  loss =  137.75185199547735
epoch  32548  loss =  137.75185167581827
epoch  32549  loss =  137.75185135617883
epoch  32550  loss =  137.75185103655903
epoch  32551  loss =  137.7518507169589
epoch  32552  loss =  137.75185039737846
epoch  32553  loss =  137.75185007781752
epoch  32554  loss =  137.7518497582763
epoch  32555  loss =  137.7518494387547
epoch  32556  loss =  137.75184911925277
epoch  32557  loss =  137.75184879977039
epoch  32558  loss =  137.75184848030761
epoch  32559  loss =  137.7518481608645
epoch  32560  loss =  137.75184784144113
epoch  32561  loss =  137.75184752203725
epoch  32562  loss =  137.751847202653
epoch  32563  loss =  137.7518468832884
epoch  32564  loss =  137.7518465639434
epoch  32565  loss =  137.75184624461806
epoch  32566  loss =  137.75184592531224
epoch  32567  loss =  137.75184560602608
epoch  32568  loss =  137.75184528675953
epoch  32569  loss =  137.75184496751268
epoch  32570  loss =  137.75184464828533
epoch  32571  loss =  13

epoch  32773  loss =  137.75178024854748
epoch  32774  loss =  137.75177993328214
epoch  32775  loss =  137.75177961803612
epoch  32776  loss =  137.75177930280935
epoch  32777  loss =  137.75177898760177
epoch  32778  loss =  137.75177867241345
epoch  32779  loss =  137.75177835724435
epoch  32780  loss =  137.75177804209451
epoch  32781  loss =  137.75177772696384
epoch  32782  loss =  137.7517774118525
epoch  32783  loss =  137.75177709676024
epoch  32784  loss =  137.75177678168728
epoch  32785  loss =  137.7517764666336
epoch  32786  loss =  137.75177615159907
epoch  32787  loss =  137.7517758365838
epoch  32788  loss =  137.75177552158766
epoch  32789  loss =  137.75177520661086
epoch  32790  loss =  137.75177489165318
epoch  32791  loss =  137.75177457671472
epoch  32792  loss =  137.75177426179556
epoch  32793  loss =  137.75177394689553
epoch  32794  loss =  137.7517736320147
epoch  32795  loss =  137.75177331715304
epoch  32796  loss =  137.75177300231067
epoch  32797  loss =

epoch  32994  loss =  137.7517110395368
epoch  32995  loss =  137.7517107284811
epoch  32996  loss =  137.75171041744426
epoch  32997  loss =  137.75171010642623
epoch  32998  loss =  137.7517097954271
epoch  32999  loss =  137.75170948444682
epoch  33000  loss =  137.75170917348532
epoch  33001  loss =  137.75170886254273
epoch  33002  loss =  137.751708551619
epoch  33003  loss =  137.75170824071407
epoch  33004  loss =  137.75170792982797
epoch  33005  loss =  137.7517076189607
epoch  33006  loss =  137.75170730811237
epoch  33007  loss =  137.75170699728284
epoch  33008  loss =  137.75170668647215
epoch  33009  loss =  137.75170637568021
epoch  33010  loss =  137.7517060649072
epoch  33011  loss =  137.75170575415294
epoch  33012  loss =  137.7517054434175
epoch  33013  loss =  137.75170513270098
epoch  33014  loss =  137.75170482200323
epoch  33015  loss =  137.75170451132428
epoch  33016  loss =  137.75170420066416
epoch  33017  loss =  137.75170389002292
epoch  33018  loss =  13

epoch  33215  loss =  137.75164275158536
epoch  33216  loss =  137.75164244465543
epoch  33217  loss =  137.751642137744
epoch  33218  loss =  137.75164183085104
epoch  33219  loss =  137.75164152397656
epoch  33220  loss =  137.75164121712064
epoch  33221  loss =  137.75164091028304
epoch  33222  loss =  137.75164060346398
epoch  33223  loss =  137.75164029666342
epoch  33224  loss =  137.75163998988137
epoch  33225  loss =  137.75163968311782
epoch  33226  loss =  137.75163937637262
epoch  33227  loss =  137.7516390696459
epoch  33228  loss =  137.75163876293774
epoch  33229  loss =  137.7516384562479
epoch  33230  loss =  137.75163814957662
epoch  33231  loss =  137.75163784292377
epoch  33232  loss =  137.7516375362894
epoch  33233  loss =  137.75163722967338
epoch  33234  loss =  137.75163692307595
epoch  33235  loss =  137.7516366164969
epoch  33236  loss =  137.7516363099364
epoch  33237  loss =  137.7516360033942
epoch  33238  loss =  137.75163569687047
epoch  33239  loss =  13

In [ ]:

w2v.predict("embedding",3)

In [ ]:

w2v.predict("deep",5)

In [ ]:
w2v.predict("jupiter",3)